pagina de referencia: https://datascienceparichay.com/article/jaccard-similarity-python/

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('tweets_2022_abril_junio.csv')

In [3]:
df.head()

,id,created_at,screen_name,text,favorite_count,retweet_count
0,1512186166438637582,2022-04-07 21:50:51 UTC,h0l4d4ni3l4,RT @ValeMirandaCC: Tras casi 50 años del golpe...,0,0
1,1512186202367045642,2022-04-07 21:51:00 UTC,Claudio70932894,RT @UTDTrabajoDigno: Mañana jueves a las 18hrs...,0,0
2,1512186287284924418,2022-04-07 21:51:20 UTC,Cesar_A_RR,RT @JaimeGuajardoR: Aquí está el aporte de @te...,0,0
3,1512186335754301446,2022-04-07 21:51:32 UTC,rosmarieher,RT @melnicksergio: la pelotudez no tiene limit...,0,0
4,1512186407841767424,2022-04-07 21:51:49 UTC,GQuelluen,RT @BSepulvedaHales: Ante la circulación de no...,0,0


In [41]:
max = 0
id = 0
for i in range(df.shape[0]):
    if len(df['text'][i]) >= max:
        max = len(df['text'][i])
        id = i

In [48]:
len(df['text'][id]), id #maximo de caracteres y su id

(203, 888457)

In [44]:
min = len(df['text'][id])
id_2 = 1
for i in range(df.shape[0]):
    if len(df['text'][i]) <= min:
        min = len(df['text'][i])
        id_2 = i

In [49]:
len(df['text'][id_2]), id_2 #minimo de caracteres y su id

(1, 4559771)